In [35]:
from tatd.utils import *
from tatd.read import *
from tatd.train import *
from tatd.tatd import *
from tatd.alternating import *
from tatd.als import *

import sys
import argparse
torch.backends.cudnn.deterministic = True
torch.manual_seed(1234)

import time
import torch.distributed as dist

import multiprocessing
import torch.multiprocessing as mp
from joblib import Parallel, delayed
from tqdm import tqdm

In [108]:
name, sparse, rank, window, penalty, opt_scheme, lr, gpu, count = 'indoor', 1, 10, 7, 0.1, 'als_adam', 0.01, 2, 10

In [109]:
loss_path, model_path, total_path  = get_path(name, sparse, rank, window,  penalty,
                                              opt_scheme, lr, count)
device = f'cuda:{gpu}'
# device ='cpu'

dataset = read_dataset(name, device)
dataset['count'], dataset['window'] = count, window
nmode, ndim, tmode =  dataset['nmode'], dataset['ndim'], dataset['tmode']

density = dataset['ts_beta']

In [110]:
model = Tatd(nmode, ndim, tmode, density, rank, window, sparse, device).to(device)

print(f"START {name} sparse : {bool(sparse)} & {opt_scheme} : {lr} "
      f"Rank : {rank} Window : {window} Penalty : {penalty} \n")

START indoor sparse : True & als_adam : 0.01 Rank : 10 Window : 7 Penalty : 0.1 



In [111]:
train, valid, test = dataset['train'], dataset['valid'], dataset['test']
nmode, tmode = dataset['nmode'], dataset['tmode']
window, count = dataset['window'], dataset['count']

dct, temp = metadata(train)
opt = optim.Adam([list(model.parameters())[tmode]], lr = lr)

In [112]:

train, valid, test = dataset['train'], dataset['valid'], dataset['test']
nmode, tmode = dataset['nmode'], dataset['tmode']
window, count = dataset['window'], dataset['count']

dct, temp = metadata(train)
if opt_scheme == 'als_adam':
    opt = optim.Adam([list(model.parameters())[tmode]], lr = lr)
else:
    opt = optim.SGD([list(model.parameters())[tmode]], lr=lr)


In [123]:

start_time = time.time()
old_rmse, inner_rmse, stop_iter, = 1e+5, 1e+5, 0
for n_iter in range(1, 1000):
    inner_num = 0
    stop = True
    for mode in range(nmode):
        tic = time.time()
        least_square(model, train, penalty, mode, dct, temp, rank, device)
        print(time.time() - tic)
        trn_rmse, trn_mae = evaluate(model, train)
        val_rmse, val_mae = evaluate(model, valid)
        print('Iter', n_iter, 'Mode', mode, 'Train', trn_rmse, 'Valid', val_rmse)
    if val_rmse > old_rmse and n_iter >1:
        stop_iter += 1
    old_rmse = val_rmse


25.792160034179688
Iter 1 Mode 0 Train 0.11333378404378891 Valid 0.8804914951324463
0.06267070770263672
Iter 1 Mode 1 Train 0.08286206424236298 Valid 0.8771814107894897
0.04667973518371582
Iter 1 Mode 2 Train 0.0825672373175621 Valid 0.8777133822441101


KeyboardInterrupt: 

In [121]:


start_time = time.time()
old_rmse, inner_rmse, stop_iter, = 1e+5, 1e+5, 0
for n_iter in range(1, 1000):
    inner_num = 0
    stop = True
    for mode in range(nmode):
        tic = time.time()
        least_square(model, train, penalty, mode, dct, temp, rank, device)
        print(time.time() - tic)
        trn_rmse, trn_mae = evaluate(model, train)
        val_rmse, val_mae = evaluate(model, valid)
        print('Iter', n_iter, 'Mode', mode, 'Train', trn_rmse, 'Valid', val_rmse)
    if val_rmse > old_rmse and n_iter >1:
        stop_iter += 1
    old_rmse = val_rmse


AttributeError: 'dict' object has no attribute 'to'

In [94]:

start_time = time.time()
old_rmse, inner_rmse, stop_iter, = 1e+5, 1e+5, 0
for n_iter in range(1, 1000):
    inner_num = 0
    stop = True
    for mode in range(nmode):
        tic = time.time()
        least_square(model, train, penalty, mode, dct, temp, rank, device)
        print(time.time() - tic)
        trn_rmse, trn_mae = evaluate(model, train)
        val_rmse, val_mae = evaluate(model, valid)
        print('Iter', n_iter, 'Mode', mode, 'Train', trn_rmse, 'Valid', val_rmse)
    if val_rmse > old_rmse and n_iter >1:
        stop_iter += 1
    old_rmse = val_rmse

def least_square(model, train, penalty, mode, dct, temp, rank, device):
    num_cores = multiprocessing.cpu_count()
    num_cores = mp.cpu_count()
    nnz_count = dct[mode]
    for idx, nnz in nnz_count.items():
        row_wise(idx, nnz, model, penalty, mode, dct, temp, rank, device)
    
    Parallel(n_jobs = num_cores)(delayed(row_wise)(idx, nnz, model, penalty, mode, 
                                                   dct, temp, rank, device)
                                for idx, nnz in nnz_count.items())
#     [pool.apply(row_wise, args = (idx, nnz, model, penalty, mode, 
#                                                    dct, temp, rank, device))
#                                 for idx, nnz in nnz_count.items()]


def row_wise(idx, nnz, model, penalty, mode, dct, temp, rank, device):
    with torch.no_grad():
        nnz_entries = temp[temp[:, mode] == idx]
        nnz_idxs = nnz_entries[:, :-1]
        nnz_vals = nnz_entries[:, -1]
    
#         ones = torch.ones((nnz, rank)).to(device)
# #         ones = [ torch.ones((nnz, rank)).to(device) * item[1][torch.LongTensor(item[0]), :] 
# #                 for i, item in enumerate(zip(nnz_idxs.T, model.factors)) 
# #                                          if i != mode]
#         for i, item in enumerate(zip(nnz_idxs.T, model.factors)):
#             if i != mode:
#                 index, factor = item
#                 ones = ones * factor[torch.LongTensor(index), :]

#         mat_b = torch.bmm(ones.view(nnz, rank, 1), ones.view(nnz, 1, rank))
#         mat_b = torch.sum(mat_b, dim=0)
#         mat_b2 = mat_b + torch.eye(rank).to(device) * penalty

#         vect_c = ones * torch.FloatTensor(nnz_vals).view(-1, 1).to(device)
#         vect_c = torch.sum(vect_c, dim=0)

#         update = torch.matmul(torch.inverse(mat_b2), vect_c)
#         update = torch.where(torch.abs(update) < 0.000001, torch.zeros_like(update), update)
#         model.factors[mode][idx] = update

In [122]:
def least_square(model, train, penalty, mode, dct, temp, rank, device):

    nnz_count = dct[mode]
    for idx, nnz in nnz_count.items():
        with torch.no_grad():
            nnz_entries = temp[temp[:, mode] == idx]
            nnz_idxs = nnz_entries[:, :-1]
            nnz_vals = nnz_entries[:, -1]

            ones = torch.ones((nnz, rank))
            for i, item in enumerate(zip(nnz_idxs.T, model.factors)):
                if i != mode:
                    index, factor = item
                    ones = ones * factor.cpu()[torch.LongTensor(index), :]

            mat_b = torch.bmm(ones.view(nnz, rank, 1), ones.view(nnz, 1, rank))
            mat_b = torch.sum(mat_b, dim=0)
            mat_b2 = mat_b + torch.eye(rank)* penalty

            vect_c = ones * torch.FloatTensor(nnz_vals).view(-1, 1)
            vect_c = torch.sum(vect_c, dim=0)

            update = torch.matmul(torch.inverse(mat_b2), vect_c)
            update = torch.where(torch.abs(update) < 0.000001, torch.zeros_like(update), update)
            model.factors[mode][idx] = update.to(device)

In [28]:
def gradient(model, opt, train, penalty, tmode):

    opt.zero_grad()
    rec = model(train.indices())
    loss = (rec - train.values()).pow(2).sum()
    loss = loss + penalty * model.smooth_reg(tmode)
#     print(f"Loss :{model.smooth_reg(tmode).data}, Penalty :{penalty}")
    loss.backward()
    opt.step()

    return loss

def metadata(train):
    indices = train.indices().cpu().numpy()
    vals = train.values().cpu().numpy()
    temp = np.vstack([indices, vals]).T

    dct = {}
    for mode, index in enumerate(indices):
        dct[mode] = {}
        unique, counts = np.unique(index, return_counts=True)
        for a, b in zip(* [unique, counts]):
            dct[mode][a] = b

    return dct, temp



In [8]:


def als_train_model(dataset, model, penalty, opt_scheme, lr, rank, device,
                    loss_path, model_path, total_path):

    train, valid, test = dataset['train'], dataset['valid'], dataset['test']
    nmode, tmode = dataset['nmode'], dataset['tmode']
    window, count = dataset['window'], dataset['count']

    dct, temp = metadata(train)
    if opt_scheme == 'als_adam':
        opt = optim.Adam([list(model.parameters())[tmode]], lr = lr)
    else:
        opt = optim.SGD([list(model.parameters())[tmode]], lr=lr)

    start_time = time.time()
    old_rmse, inner_rmse, stop_iter, = 1e+5, 1e+5, 0
    for n_iter in range(1, 1000):
        inner_num = 0
        stop = True
        for mode in range(0, nmode):
#             if mode != tmode:
            tic = time.time()
            least_square(model, train, penalty, mode, dct, temp, rank, device)
            print(time.time() - tic)
            trn_rmse, trn_mae = evaluate(model, train)
            val_rmse, val_mae = evaluate(model, valid)
            print('Iter', n_iter, 'Mode', mode, 'Train', trn_rmse, 'Valid', val_rmse)
#             else:
#                 model.turn_on_grad(mode)
#                 while(stop):
#                     stop = gradient(model, opt, train, penalty, tmode)
#                     trn_rmse, trn_mae = evaluate(model, train)
#                     val_rmse, val_mae = evaluate(model, valid)
#                     #print('Iter', n_iter, 'Mode', mode, 'Train', trn_rmse, 'Valid', val_rmse)
#                     if inner_num > 10:
#                         stop = False
#                     if val_rmse > inner_rmse:
#                         # break
#                         inner_num += 1
#                     inner_rmse = val_rmse
#                     if isNaN(trn_rmse):
#                         print("Nan break")
#                         break
#             trn_rmse, trn_mae = evaluate(model, train)
#             val_rmse, val_mae = evaluate(model, valid)

        if val_rmse > old_rmse and n_iter >1:
            stop_iter += 1
        old_rmse = val_rmse


In [7]:
import time
import torch
import torch.functional as F
import torch.optim as optim
import torch.nn as nn

from tqdm import tqdm
from tqdm import trange
from tatd.tatd import *
from tatd.utils import *
from tatd.train import *

def gradient(model, opt, train, penalty, tmode):

    opt.zero_grad()
    rec = model(train.indices())
    loss = (rec - train.values()).pow(2).sum()
    loss = loss + penalty * model.smooth_reg(tmode)
#     print(f"Loss :{model.smooth_reg(tmode).data}, Penalty :{penalty}")
    loss.backward()
    opt.step()

    return loss

def metadata(train):
    indices = train.indices().cpu().numpy()
    vals = train.values().cpu().numpy()
    temp = np.vstack([indices, vals]).T

    dct = {}
    for mode, index in enumerate(indices):
        dct[mode] = {}
        unique, counts = np.unique(index, return_counts=True)
        for a, b in zip(* [unique, counts]):
            dct[mode][a] = b

    return dct, temp

def least_square(model, train, penalty, mode, dct, temp, rank, device):

    nnz_count = dct[mode]
    for idx, nnz in nnz_count.items():
        with torch.no_grad():
            nnz_entries = temp[temp[:, mode] == idx]
            nnz_idxs = nnz_entries[:, :-1]
            nnz_vals = nnz_entries[:, -1]

            ones = torch.ones((nnz, rank)).to(device)
            for i, item in enumerate(zip(nnz_idxs.T, model.factors)):
                if i != mode:
                    index, factor = item
                    ones = ones * factor[torch.LongTensor(index).to(device), :]

            mat_b = torch.bmm(ones.view(nnz, rank, 1), ones.view(nnz, 1, rank))
            mat_b = torch.sum(mat_b, dim=0)
            mat_b2 = mat_b + torch.eye(rank).to(device) * penalty

            vect_c = ones * torch.FloatTensor(nnz_vals).view(-1, 1).to(device)
            vect_c = torch.sum(vect_c, dim=0)

            update = torch.matmul(torch.inverse(mat_b2), vect_c)
            update = torch.where(torch.abs(update) < 0.000001, torch.zeros_like(update), update)
            model.factors[mode][idx] = update

def als_train_model(dataset, model, penalty, opt_scheme, lr, rank, device,
                    loss_path, model_path, total_path):

    train, valid, test = dataset['train'], dataset['valid'], dataset['test']
    nmode, tmode = dataset['nmode'], dataset['tmode']
    window, count = dataset['window'], dataset['count']

    head = 'Iters\tTime\tTrnRMSE\tTrMAE\tValRMSE\tValMAE\n'
    with open(loss_path, 'w') as f:
        f.write(head)

    dct, temp = metadata(train)
    if opt_scheme == 'als_adam':
        opt = optim.Adam([list(model.parameters())[tmode]], lr = lr)
    else:
        opt = optim.SGD([list(model.parameters())[tmode]], lr=lr)

    start_time = time.time()
    old_rmse, inner_rmse, stop_iter, = 1e+5, 1e+5, 0
    for n_iter in trange(1, 1000):
        inner_num = 0
        stop = True
        for mode in range(nmode):
            if mode != tmode:
                least_square(model, train, penalty, mode, dct, temp, rank, device)
                trn_rmse, trn_mae = evaluate(model, train)
                val_rmse, val_mae = evaluate(model, valid)
                print('Iter', n_iter, 'Mode', mode, 'Train', trn_rmse, 'Valid', val_rmse)
            else:
                model.turn_on_grad(mode)
                while(stop):
                    stop = gradient(model, opt, train, penalty, tmode)
                    trn_rmse, trn_mae = evaluate(model, train)
                    val_rmse, val_mae = evaluate(model, valid)
                    #print('Iter', n_iter, 'Mode', mode, 'Train', trn_rmse, 'Valid', val_rmse)
                    if inner_num > 10:
                        stop = False
                    if val_rmse > inner_rmse:
                        # break
                        inner_num += 1
                    inner_rmse = val_rmse
                    if isNaN(trn_rmse):
                        print("Nan break")
                        break
            trn_rmse, trn_mae = evaluate(model, train)
            val_rmse, val_mae = evaluate(model, valid)

        if val_rmse > old_rmse and n_iter >1:
            stop_iter += 1
        old_rmse = val_rmse

        with open(loss_path, 'a') as f:
            elapsed = time.time() - start_time
            f.write(f'{n_iter:5d}\t{elapsed:.5f}\t')
            f.write(f'{trn_rmse:.5f}\t{trn_mae:.5f}\t')
            f.write(f'{val_rmse:.5f}\t{val_mae:.5f}\n')
        if stop_iter == 1 or n_iter == 999:
            te_rmse, te_mae = evaluate(model, test)
            with open(total_path, 'a') as f1:
                f1.write(f'{count}\t{n_iter:5d}\t{elapsed:.3f}\t{model.sparse}\t')
                f1.write(f'{model.factors[0].shape[1]:2d}\t')
                f1.write(f'{window:2d}\t{penalty:.3f}\t')
                f1.write(f'{opt_scheme}\t{lr:5f}\t')
                f1.write(f'{te_rmse:.5f}\t{te_mae:.5f}\n')

            p = f'{model_path}-{n_iter}.pth.tar'
            save_checkpoints(model, p)
            break
